This notebook generates and runs queries to download Gaia data for the region around the GD-1 stream.

Note that we don't use the Gaia-PS1 cross-match because of the aggressive quality cuts the Gaia team applied when loading the PS1 data, so here we only download Gaia data and later cross-match to PS1 using LSD.

In [ ]:
import pathlib
import warnings

# Third-party
import astropy.coordinates as coord
import astropy.table as at
from astropy.io import fits
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import gala.coordinates as gc
import gala.dynamics as gd
from pyia import GaiaData
from astroquery.gaia import Gaia
Gaia.login(credentials_file='/Users/apricewhelan/.gaia/archive.login')

In [ ]:
data_path = pathlib.Path('../data/gd1-polygon/').resolve()
data_path.mkdir(exist_ok=True, parents=True)

In [ ]:
gaia_cols = ['source_id', 
             'ra', 'dec', 'parallax', 'parallax_error', 
             'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_parallax_corr', 'ra_pmra_corr', 
             'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 
             'parallax_pmdec_corr', 'pmra_pmdec_corr', 'visibility_periods_used', 
             'phot_g_mean_mag', 'phot_g_mean_flux_over_error', 
             'phot_bp_mean_mag', 'phot_bp_mean_flux_over_error', 
             'phot_rp_mean_mag', 'phot_rp_mean_flux_over_error', 
             'phot_bp_rp_excess_factor', 'astrometric_chi2_al', 'astrometric_n_good_obs_al']

In [ ]:
q_base ='''SELECT {0}
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -0.75 AND bp_rp < 2 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
                       {1[0].ra.degree}, {1[0].dec.degree}, 
                       {1[1].ra.degree}, {1[1].dec.degree}, 
                       {1[2].ra.degree}, {1[2].dec.degree}, 
                       {1[3].ra.degree}, {1[3].dec.degree})) = 1
'''

In [ ]:
queries = []
jobs = []
for l in np.arange(-100, 20, 10):
    print(l)
    
    fn = path.join(data_path, 'gd1_{0:.0f}.fits'.format(l))
    if path.exists(fn):
        print('{0} exists...skipping'.format(fn))
        continue

    corners = gc.GD1(phi1=[l, l, l+10, l+10]*u.deg, 
                     phi2=[-10, 5, 5, -10]*u.deg)
    corners_icrs = corners.transform_to(coord.ICRS)
    q = q_base.format(', '.join(gaia_cols), corners_icrs)
        
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        job = Gaia.launch_job_async(q, name='GD1-{0}'.format(l), 
                                    background=True)
    jobs.append(job)

In [ ]:
if jobs:
    for job in jobs:
        tbl = job.get_results()

        for c in tbl.colnames: # hack to make sure object arrays are string
            if tbl[c].dtype == object:
                tbl[c] = np.array(tbl[c]).astype(str)
        tbl.write(fn, overwrite=True)

# Combine tables

In [ ]:
from numpy.lib.recfunctions import stack_arrays

In [ ]:
all_filename = '../data/gd1-all.fits'
if not path.exists(all_filename):
    arrs = []
    for filename in data_path.glob('*.fits'):
        arrs.append(np.array(fits.getdata(filename)))
    arr = stack_arrays(arrs, asrecarray=True, usemask=False)
    t = Table(arr)
    t.write(all_filename)

In [ ]:
g = GaiaData(all_filename)

In [ ]:
c_gd1 = g.get_skycoord(distance=8*u.kpc, radial_velocity=0*u.km/u.s).transform_to(gc.GD1)
c_gd1 = gc.reflex_correct(c_gd1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(c_gd1.phi1.wrap_at(180*u.deg),
        c_gd1.phi2, 
        marker=',', linestyle='none', alpha=0.05)

ax.set_xlim(-100, 20)
ax.set_ylim(-10, 5)

ax.set_aspect('equal')

In [ ]:
phi_mask = ((np.abs(c_gd1.phi2) < 1*u.deg) & 
            (c_gd1.phi1.wrap_at(180*u.deg) > -60*u.deg) & 
            (c_gd1.phi1.wrap_at(180*u.deg) < 10*u.deg))

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(c_gd1.pm_phi1_cosphi2[phi_mask], 
        c_gd1.pm_phi2[phi_mask], 
        marker='.', alpha=0.15, ls='none')
ax.set_xlim(-12, 8)
ax.set_ylim(-10, 10)